#### 加载数据

transformers==4.40.0

torch==1.11.0+cu113

torch-geometric==2.5.2

torch-scatter==2.0.9

torch-sparse==0.6.13

In [6]:
import pickle
with open("/root/autodl-tmp/graph_data/mix_train.pkl", "rb") as f:
    hc3_train = pickle.load(f)
with open("/root/autodl-tmp/graph_data/mix_val.pkl", "rb") as f:
    hc3_val = pickle.load(f)

#### 训练模型

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
from datetime import datetime
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime
from tqdm import tqdm

# 构建 GCN 模型
class GCN2(nn.Module):
    def __init__(self,  input_dim, hidden_dim, output_dim):
        super(GCN2, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, output_dim)
        self.fc = nn.Linear(output_dim, 1) 
        self.dropout = nn.Dropout(0.5)
        
    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.dropout(x)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = self.dropout(x)
        x = self.fc(x)
        x = torch.mean(x, dim=0, keepdim=True)  
        return torch.sigmoid(x)  

class GCN4(nn.Module):
    def __init__(self,  input_dim, hidden_dim, hidden_dim2, hidden_dim3, output_dim):
        super(GCN4, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, hidden_dim2)
        self.conv3 = GCNConv(hidden_dim2, hidden_dim3)
        self.conv4 = GCNConv(hidden_dim3, output_dim)
        self.fc = nn.Linear(output_dim, 1) 
        self.dropout = nn.Dropout(0.5)
        
    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.dropout(x)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = self.dropout(x)
        x = self.conv3(x, edge_index)
        x = F.relu(x)
        x = self.dropout(x)
        x = self.conv4(x, edge_index)
        x = F.relu(x)
        x = self.dropout(x)
        x = self.fc(x)
        x = torch.mean(x, dim=0, keepdim=True)  
        return torch.sigmoid(x)  


class GCNMulticlass(nn.Module):
    def __init__(self, input_dim, hidden_dim1, hidden_dim2, num_classes):
        super(GCNMulticlass, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim1)
        self.conv2 = GCNConv(hidden_dim1, hidden_dim2)  
        self.fc = nn.Linear(hidden_dim2, num_classes)  
        self.dropout = nn.Dropout(0.5)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.dropout(x)
        
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = self.dropout(x)
        
        x = torch.mean(x, dim=0, keepdim=True)  
        x = self.fc(x)
        return F.log_softmax(x, dim=1) 

In [8]:
seed = 2024
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed) 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [9]:
input_dim = 768  # 输入维度
hidden_dim = 512  # 隐藏层维度
hidden_dim2 = 256  # 隐藏层维度
hidden_dim3 = 128  # 隐藏层维度
output_dim = 64  # 输出类别数
# gcnmodel = GCN2(input_dim, hidden_dim2, output_dim)
# gcnmodel = GCN4(input_dim, hidden_dim, hidden_dim2, hidden_dim3, output_dim)
gcnmodel = GCNMulticlass(input_dim, hidden_dim2, output_dim, 3)
optimizer = optim.Adam(gcnmodel.parameters(), lr=0.0001)
criterion = nn.BCELoss()
CE_criterion = nn.CrossEntropyLoss()

In [10]:
train_len = len(hc3_train['y'])
val_len = len(hc3_val['y'])
epochs = 30
train_loss = []
val_loss = []
train_acc = []
val_acc = []
val_max_acc = -1
writer = SummaryWriter('logs/mix'+ datetime.now().strftime("%Y%m%d-%H%M%S"))
for epoch in range(epochs):
    # 训练集
    gcnmodel.train()
    epoch_loss = 0.0
    correct_predictions = 0
    for i in tqdm(range(train_len),  f"epoch: {epoch+1}, Training"):
        data = Data(x=hc3_train['all_token_embeddings'][i], edge_index=hc3_train['all_edge_index'][i], y=hc3_train['y'][i])
        optimizer.zero_grad()
        outputs = gcnmodel(data)
        # loss = criterion(outputs, data.y.float().view(-1, 1))
        loss = CE_criterion(outputs, data.y.long().view(-1, 1).squeeze(0))
        # print(loss)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        _, predictions = torch.max(outputs, 1) 
        correct_predictions += (predictions == data.y).sum().item()
    epoch_loss /= train_len
    writer.add_scalar('Loss/train', epoch_loss, epoch)
    epoch_acc = correct_predictions / train_len
    writer.add_scalar('Acc/train', epoch_acc, epoch)
    print(f"epoch: {epoch+1}, train_loss: {epoch_loss}, train_acc: {epoch_acc}")
    train_loss.append(epoch_loss)
    train_acc.append(epoch_acc)
    
    # 验证集
    gcnmodel.eval()
    epoch_loss = 0.0
    correct_predictions = 0
    all_predictions = []
    with torch.no_grad():
        for i in tqdm(range(val_len),  f"epoch: {epoch+1}, Validation"):
            data = Data(x=hc3_val['all_token_embeddings'][i], edge_index=hc3_val['all_edge_index'][i], y=hc3_val['y'][i])
            outputs = gcnmodel(data)
            # loss = criterion(outputs, data.y.float().view(-1, 1))
            loss = CE_criterion(outputs, data.y.long().view(-1, 1).squeeze(0))
            epoch_loss += loss.item()
            _, predictions = torch.max(outputs, 1) 
            correct_predictions += (predictions == data.y).sum().item()
            all_predictions.append(predictions)
    epoch_loss /= val_len
    writer.add_scalar('Loss/val', epoch_loss, epoch)
    epoch_acc = correct_predictions / val_len
    writer.add_scalar('Acc/val', epoch_acc, epoch)
    print(f"epoch: {epoch+1}, val_loss: {epoch_loss}, val_acc: {epoch_acc}")
    val_loss.append(epoch_loss)
    val_acc.append(epoch_acc)

    if epoch_acc >= val_max_acc:
        val_max_acc = epoch_acc
        torch.save(gcnmodel.state_dict(), './model/mix_gcn_model.pth')

writer.close()

epoch: 1, Training: 100%|██████████| 4800/4800 [00:33<00:00, 143.48it/s]


epoch: 1, train_loss: 0.2994169792495111, train_acc: 0.8908333333333334


epoch: 1, Validation: 100%|██████████| 750/750 [00:01<00:00, 571.13it/s]


epoch: 1, val_loss: 0.21089619938431134, val_acc: 0.9413333333333334


epoch: 2, Training: 100%|██████████| 4800/4800 [00:33<00:00, 141.71it/s]


...

epoch: 29, val_loss: 0.25695445893220825, val_acc: 0.9493333333333334


epoch: 30, Training: 100%|██████████| 4800/4800 [00:34<00:00, 139.70it/s]


epoch: 30, train_loss: 0.006575552438553746, train_acc: 0.9979166666666667


epoch: 30, Validation: 100%|██████████| 750/750 [00:01<00:00, 580.38it/s]

epoch: 30, val_loss: 0.18147696985641032, val_acc: 0.9613333333333334


#### 测试

In [12]:
import pickle
test_file = "mix_test"
with open(f"/root/autodl-tmp/graph_data/{test_file}.pkl", "rb") as f:
    hc3_test = pickle.load(f)
test_len = len(hc3_test['y'])

In [24]:
from sklearn.metrics import roc_auc_score, f1_score

test_gcnmodel = GCNMulticlass(input_dim, hidden_dim2, output_dim, 3)
# test_gcnmodel = gcnmodel
test_gcnmodel.load_state_dict(torch.load('./model/mix_gcn_model.pth'))
test_gcnmodel.eval()
test_loss = 0.0
correct_predictions = 0
y_pred = list()
with torch.no_grad():
    for i in tqdm(range(test_len),  f"Test"):
        data = Data(x=hc3_test['all_token_embeddings'][i], edge_index=hc3_test['all_edge_index'][i], y=hc3_test['y'][i])
        outputs = test_gcnmodel(data)
        loss = CE_criterion(outputs, data.y.long().view(-1, 1).squeeze(0))
        test_loss += loss.item()
        _, predictions = torch.max(outputs, 1) 
        correct_predictions += (predictions == data.y).sum().item()
        y_pred.append(predictions.item())
y_true = hc3_test['y']
test_loss /= test_len
test_acc = correct_predictions / test_len
test_f1 = f1_score(y_true, y_pred, average='weighted')
print(f"test_loss: {test_loss}, test_acc: {test_acc}, test_f1: {test_f1}")

Test: 100%|██████████| 750/750 [00:01<00:00, 570.34it/s]

test_loss: 0.12448984051934489, test_acc: 0.9693333333333334, test_f1: 0.9693918394675536


In [25]:
with open(f"/root/autodl-tmp/result/test_result.txt", "a", encoding="utf-8") as w:
    w.write(f"{test_file}\tacc: {test_acc}\tf1: {test_f1}\t{datetime.now()}\n")